In [2]:
import pandas as pd

# Data Cleaning Script for CHRP-AI

### Get data to be cleaned

In [34]:
# filepath to data to be cleaned
filepath = '/Users/ethanweiss/Desktop/Class documents/Spring 2020/Senior Design/Chiller Tons/TONS/AHSC201718.csv'
# read data from csv into pandas dataframe
data = pd.read_csv(filepath)

In [37]:
data.head()

,Date,UAFM.AHSC.9002,Status,UAFM.AHSC.9006,Status.1,UAFM.AHSC.9010,Status.2,UAFM.AHSC.9014,Status.3,UAFM.AHSC.T9002,Status.4,UAFM.AHSC.T9003,Status.5
0,2/20/2017 12:00:00 AM,0.0,UNRELIABLE,1062.966,UNRELIABLE,0.0,UNRELIABLE,0.0,UNRELIABLE,2304.641,OK,0.0,OK
1,2/20/2017 12:02:00 AM,0.0,UNRELIABLE,1062.966,UNRELIABLE,0.0,UNRELIABLE,0.0,UNRELIABLE,2251.096,OK,0.0,OK
2,2/20/2017 12:04:00 AM,0.0,UNRELIABLE,1062.966,UNRELIABLE,0.0,UNRELIABLE,0.0,UNRELIABLE,2262.923,OK,0.0,OK
3,2/20/2017 12:06:00 AM,0.0,UNRELIABLE,1062.966,UNRELIABLE,0.0,UNRELIABLE,0.0,UNRELIABLE,2236.943,OK,0.0,OK
4,2/20/2017 12:08:00 AM,0.0,UNRELIABLE,1062.966,UNRELIABLE,0.0,UNRELIABLE,0.0,UNRELIABLE,2318.157,OK,0.0,OK


In [25]:
mapper = pd.read_csv('/Users/ethanweiss/Desktop/Class documents/Spring 2020/Senior Design/IDmap.csv',header = None)
header_dict = {}
for (x,y) in zip(list(mapper.iloc[:,0]),list(mapper.iloc[:,1])):
    header_dict[x] = y

In [15]:
header_dict

{'UAFM.AHSC.9003': 'AHSC_CHL01_KW',
 'UAFM.AHSC.9007': 'AHSC_CHL02_KW',
 'UAFM.AHSC.9011': 'AHSC_CHL03_KW',
 'UAFM.AHSC.9015': 'AHSC_CHL04_KW',
 'UAFM.AHSC.T9000': 'AHSC_CHL07_KW',
 'UAFM.AHSC.T9001': 'AHSC_CHL08_KW',
 'UAFM.AHSC.9002': 'AHSC_CHL01_TONS',
 'UAFM.AHSC.9006': 'AHSC_CHL02_TONS',
 'UAFM.AHSC.9010': 'AHSC_CHL03_TONS',
 'UAFM.AHSC.9014': 'AHSC_CHL04_TONS',
 'UAFM.AHSC.T9002': 'AHSC_CHL07_TONS',
 'UAFM.AHSC.T9003': 'AHSC_CHL08_TONS',
 'UAFM.CHRP.9002': 'CHRP_CHL06_TONS',
 'UAFM.CHRP.9007': 'CHRP_CHL07_TONS',
 'UAFM.CHRP.9012': 'CHRP_CHL08_TONS',
 'UAFM.CHRP.9017': 'CHRP_CHL09_TONS',
 'UAFM.CHRP.9022': 'CHRP_CHL10_TONS',
 'UAFM.CHRP.9030': 'CHRP_CHL11_TONS',
 'UAFM.CHRP.9038': 'CHRP_CHL12_TONS',
 'UAFM.CHRP.9003': 'CHRP_CHL06_KW',
 'UAFM.CHRP.9008': 'CHRP_CHL07_KW',
 'UAFM.CHRP.9013': 'CHRP_CHL08_KW',
 'UAFM.CHRP.9018': 'CHRP_CHL09_KW',
 'UAFM.CHRP.9023': 'CHRP_CHL10_KW',
 'UAFM.CHRP.9032': 'CHRP_CHL11_KW',
 'UAFM.CHRP.9039': 'CHRP_CHL12_KW',
 'UAFM.CRB.9038': 'CRB_CHL01B_KW',

In [26]:
data.rename(header_dict,axis = 1,inplace = True)

In [27]:
# initialize dictionary for unreliable indecies
unreliable_dict = {}

### Get index of UNRELIABLE points
##### Store indecies of unreliable data points in dictionary where keys are column indecies

In [28]:
# get indecies for all "eDNA Status as String" columns
status_cols = [i for i in range(len(data.columns)) if i%3 == 2]
# chiller_cols = [i for i in range(len(data.columns)) if i%3 == 0]
# chiller_col_labels = [data.columns[x] for x in chiller_cols]

# Loop through all status columns 
for j in status_cols:
    unr_index = [] # reset unreliable index list to empty
    for i in range(len(data)): # loop through all rows of data
        if data.iloc[i,j] == 'UNRELIABLE': # check if unreliable
            unr_index.append(i) # if unreliable, add index to list
    unreliable_dict[j] = unr_index # assign list of unreliable indecies to column index key

In [29]:
# DEBUG
# Count number of UNRELIABLE points
total = 0
for x in unreliable_dict.keys():
    total += len(unreliable_dict[x])
print(total)

5295


### **Use Linear Interpolation to get values for previously UNRELIABLE data points**
Assuming that UNRELIABLE data points occur in bunches, we want to linearly interpolate using the boundary values of Reliable data points. The "before bound" will be the value of the Reliable data point which occured right before the bunch of UNRELIABLE data points. The "after bound" will be the value of the Reliable data point which occurs right after the bunch of UNRELIABLE data points.

Process for Linear Interpolation: <br>
    1) Find "before index" --> Index of previous Reliable data point <br>
    2) Find "before bound" --> Value of previous Reliable data point <br>
    3) Find "after index" --> Index of next Reliable data point <br>
    4) Find "after bound" --> Value of next Reliable data point <br>
    5) Calculate slope --> $slope = \frac{a_{bound}-b_{bound}}{a_{index}-b_{index}}$ <br>
    6) Calculate new value --> $b_{bound} + slope * (c_{index} - b_{index})$ <br> 


In [47]:
# loop through all column indecies with unreliable data
for x in unreliable_dict.keys():
    # loop through all unreliable values
    for i in unreliable_dict[x]:
        # loop through all indecies before unreliable point
        for j in range(1,i):
            if data.iloc[i-j,x] != 'UNRELIABLE': # check for unreliable point
                # IF NOT UNRELIABLE: 
                before_bound = data.iloc[i-j,x-1] # set before bound value
                before_index = i-j # set before index value
                break # break loop once before bound is found
        # loop through all indecies after unreliable point
        for j in range(1,len(data) - i): 
            if data.iloc[i+j,x] != 'UNRELIABLE': # check for unreliable point
                # IF NOT UNRELIABLE:
                after_bound = data.iloc[i+j,x-1] # set after bound value
                after_index = i+j # set after index value
                break # break loop once after bound is found
       
        # Linear Interpolation to substitute for UNRELIABLE values
        slope = (after_bound - before_bound)/(after_index - before_index)
        data.iloc[i,x-1] = before_bound + slope * (i - before_index)

# Replace 'UNRELIABLE' tags with 'LI' tags
for x in unreliable_dict.keys():
    for i in unreliable_dict[x]:
        data.iloc[i,x] = 'LI'
        
        # DEBUG
#         if (x == 32 and i == 354):
#             print('before bound: {0}'.format(before_bound))
#             print('before index: {0}'.format(before_index))
#             print('after bound: {0}'.format(after_bound))
#             print('after index: {0}'.format(after_index))
#             print('slope: {0}'.format(slope))

In [49]:
# DEBUG
# Count number of LI tags
total = 0
for j in status_cols:
    for i in range(len(data)):
        if data.iloc[i,j] == 'LI':
            total += 1
print(total)

2951


### Save dataset with 'LI' tags as CSV

In [50]:
li_filepath = filepath.split('.csv')[0] + '_LItags.csv'
data.to_csv(li_filepath,index = False)

### Only keep relevant columns and rename accordingly
Keep only the first Date/time column and all Values columns. Rename Values columns according to chiller name.

In [ ]:
data = pd.read_csv(li_filepath)

In [27]:
data.head()

,Date,AHSC_CHL01_KW,Status,AHSC_CHL02_KW,Status.1,AHSC_CHL03_KW,Status.2,AHSC_CHL04_KW,Status.3,AHSC_CHL07_KW,Status.4,AHSC_CHL08_KW,Status.5
0,2/20/2017 12:00:00 AM,0.0,OK,0.0,OK,0.0,OK,0.0,OK,1229,OK,0,OK
1,2/20/2017 12:02:00 AM,0.0,OK,0.0,OK,0.0,OK,0.0,OK,1201,OK,0,OK
2,2/20/2017 12:04:00 AM,0.0,OK,0.0,OK,0.0,OK,0.0,OK,1201,OK,0,OK
3,2/20/2017 12:06:00 AM,0.0,OK,0.0,OK,0.0,OK,0.0,OK,1219,OK,0,OK
4,2/20/2017 12:08:00 AM,0.0,OK,0.0,OK,0.0,OK,0.0,OK,1228,OK,0,OK


In [51]:
# Get chiller names
# chiller_cols = [i for i in range(len(data.columns)) if i%3 == 0]
# chiller_cols = [data.columns[x] for x in chiller_cols]

# Get chiller names for second data pull
chiller_cols = [i * 2+1 for i in range(int(len(data.columns)/2))]
chiller_cols = [data.columns[x] for x in chiller_cols]

# Get value columns to keep
# value_cols = [i for i in range(len(data.columns)) if i%3 == 1]
# value_cols = [data.columns[x] for x in value_cols]

# Compile columns to keep in dataframe
# data.rename({'AHSC_CHL01_ChlrPwr':'Date/Time'},axis = 1,inplace = True)
# keep_cols = ['Date/Time'] + value_cols

# Get keep cols for second data pull
keep_cols = ['Date'] + chiller_cols

# Get relevant subset of dataframe
rel_data = data[keep_cols]
# Create Dictionary for guide to rename columns with appropriate chiller names
# rename_dict = {}
# for i in range(len(keep_cols[1:])):
#     rename_dict[keep_cols[i+1]] = chiller_cols[i]
# # Assign new column names (appropriate chiller names)
# rel_data.rename(rename_dict,axis = 1,inplace = True)

In [52]:
rel_data.head()

,Date,AHSC_CHL01_KW,AHSC_CHL02_KW,AHSC_CHL03_KW,AHSC_CHL04_KW,AHSC_CHL07_KW,AHSC_CHL08_KW
0,2/20/2018 12:00:00 AM,0.0,0.0,0.0,0.0,1145,0
1,2/20/2018 12:02:00 AM,0.0,0.0,0.0,0.0,1171,0
2,2/20/2018 12:04:00 AM,0.0,0.0,0.0,0.0,1158,0
3,2/20/2018 12:06:00 AM,0.0,0.0,0.0,0.0,1157,0
4,2/20/2018 12:08:00 AM,0.0,0.0,0.0,0.0,1078,0


## Get Chiller Values and Status tags (for color plotting)

In [50]:
filepath = '/Users/ethanweiss/Desktop/Class documents/Spring 2020/Senior Design/Chiller Power/Power_Data_Full_LItags.csv'
data = pd.read_csv(filepath)

In [51]:
# Get chiller names
chiller_cols = [i for i in range(len(data.columns)) if i%3 == 0]
chiller_cols = [data.columns[x] for x in chiller_cols]
# Get status names
status_cols = [i for i in range(len(data.columns)) if i%3 == 2]
status_cols = [data.columns[x] for x in status_cols]
# Get value columns to keep
value_cols = [i for i in range(len(data.columns)) if i%3 == 1]
value_cols = [data.columns[x] for x in value_cols]
# Compile columns to keep in dataframe
data.rename({'AHSC_CHL01_ChlrPwr':'Date/Time'},axis = 1,inplace = True)
keep_cols = ['Date/Time']
for i in range(len(chiller_cols)):
    keep_cols.append(value_cols[i])
    keep_cols.append(status_cols[i])
# Get relevant subset of dataframe
rel_data = data[keep_cols].copy()
# Create Dictionary for guide to rename columns with appropriate chiller names
rename_dict = {}
for i in range(int(len(keep_cols[1:])/2)):
    rename_dict[keep_cols[i*2+1]] = chiller_cols[i]
# Assign new column names (appropriate chiller names)
rel_data.rename(rename_dict,axis = 1,inplace = True)

In [53]:
color_fp = filepath.split('_LI')[0] + '_ValuesAndStatus.csv'
rel_data.to_csv(color_fp,index = False)

In [56]:
rel_data.head()

,Date/Time,AHSC_CHL01_ChlrPwr,eDNA Status as String1,AHSC_CHL02_ChlrPwr,eDNA Status as String2,AHSC_CHL03_ChlrPwr,eDNA Status as String3,AHSC_CHL04_ChlrPwr,eDNA Status as String4,AHSC_CHL07_ChlrPwr,...,CHRP_CHL07_ChlrPwr,eDNA Status as String14,CHRP_CHL08_ChlrPwr,eDNA Status as String15,CHRP_CHL09_ChlrPwr,eDNA Status as String16,CHRP_CHL11_ChlrPwr,eDNA Status as String17,CHRP_CHL12_ChlrPwr,eDNA Status as String18
0,2019-02-12 00:00:00,0.0,OK,0.0,OK,0.0,OK,556.2,OK,0.0,...,0.0,OK,0.0,OK,0.0,OK,1020.9,OK,0.0,OK
1,2019-02-12 00:02:00,0.0,OK,0.0,OK,0.0,OK,566.1,OK,0.0,...,0.0,OK,0.0,OK,0.0,OK,1020.9,OK,0.0,OK
2,2019-02-12 00:04:00,0.0,OK,0.0,OK,0.0,OK,572.0,OK,0.0,...,0.0,OK,0.0,OK,0.0,OK,1018.3,OK,0.0,OK
3,2019-02-12 00:06:00,0.0,OK,0.0,OK,0.0,OK,577.9,OK,0.0,...,0.0,OK,0.0,OK,0.0,OK,1012.8,OK,0.0,OK
4,2019-02-12 00:08:00,0.0,OK,0.0,OK,0.0,OK,568.8,OK,0.0,...,0.0,OK,0.0,OK,0.0,OK,1007.4,OK,0.0,OK


In [ ]:
# set color values
for x in status_cols:
    for i in range(len(rel_data[x])):
        if rel_data.loc[i,x] == 'OK':
            rel_data.loc[i,x] = 'blue'
        else:
            rel_data.loc[i,x] = 'red'

In [ ]:
color_fp = filepath.split('_LI')[0] + '_ValuesAndColors.csv'
rel_data.to_csv(color_fp,index = False)

### Save CSV of relevant data

In [53]:
rel_filepath = filepath.split('.csv')[0] + '_ChillerValuesOnly.csv'
rel_data.to_csv(rel_filepath,index = False)

In [54]:
rel_data.head()

,Date,AHSC_CHL01_KW,AHSC_CHL02_KW,AHSC_CHL03_KW,AHSC_CHL04_KW,AHSC_CHL07_KW,AHSC_CHL08_KW
0,2/20/2018 12:00:00 AM,0.0,0.0,0.0,0.0,1145,0
1,2/20/2018 12:02:00 AM,0.0,0.0,0.0,0.0,1171,0
2,2/20/2018 12:04:00 AM,0.0,0.0,0.0,0.0,1158,0
3,2/20/2018 12:06:00 AM,0.0,0.0,0.0,0.0,1157,0
4,2/20/2018 12:08:00 AM,0.0,0.0,0.0,0.0,1078,0


### Find Outliers
Use 1 hour interval (15 points before and 15 points after current point)

For points with only N points before (w/ N < 15), use N points before and 30-N points after.<br>
For points with only N points after (w/ N < 15), use N points after and 30-N points before.

In [ ]:
outlier_cols = []
for x in rel_data.columns[1:]:
    outlier_col = [0 for i in range(len(rel_data))]
    for i in range(len(rel_data)):
        if i < 15:
            window = rel_data.loc[0:29,x]
            q1 = window.quantile(0.25)
            #print(q1)
            q3 = window.quantile(0.75)
            iqr = abs(q3 - q1)
            if ((rel_data.loc[i,x] < q1 - 1.5 * iqr) or (rel_data.loc[i,x] > q3 + 1.5 * iqr)):
                outlier_col[i] = 1
        elif i > len(rel_data) - 15:
            window = rel_data.loc[len(rel_data) - 30:len(rel_data) - 1,x]
            q1 = window.quantile(0.25)
            q3 = window.quantile(0.75)
            iqr = abs(q3 - q1)
            if ((rel_data.loc[i,x] < q1 - 1.5 * iqr) or (rel_data.loc[i,x] > q3 + 1.5 * iqr)):
                outlier_col[i] = 1
        else:
            window = rel_data.loc[i-15:i+14,x]
            q1 = window.quantile(0.25)
            q3 = window.quantile(0.75)
            iqr = abs(q3 - q1)
            if ((rel_data.loc[i,x] < q1 - 1.5 * iqr) or (rel_data.loc[i,x] > q3 + 1.5 * iqr)):
                outlier_col[i] = 1
    outlier_cols.append(outlier_col)

In [204]:
rel_data.columns

Index(['Date/Time', 'AHSC_CHL01_TONS', 'AHSC_CHL02_TONS', 'AHSC_CHL03_TONS',
       'CHRP_CHL08_TONS', 'CHRP_CHL09_TONS', 'CHRP_CHL06_TONS',
       'CHRP_CHL07_TONS', 'CHRP_CHL10_TONS', 'CHRP_CHL11_TONS',
       'CHRP_CHL12_TONS', 'CRB_CHL05_TONS', 'CRB_CHL06_TONS', 'CRB_CHL07_TONS',
       'CRB_CHL08_TONS', 'CRB_CHL09_TONS', 'CRB_CHL10_TONS', 'CRB_CHL01B_TONS',
       'AHSC_CHL04_TONS', 'AHSC_CHL07_TONS', 'AHSC_CHL08_TONS',
       'CRB_CHL02B_TONS', 'CRB_CHL03B_TONS'],
      dtype='object')

## Full function to perform data cleaning

In [38]:
def dataCleaning(filepath):
    # read data from csv into pandas dataframe
    data = pd.read_csv(filepath)
    
    # for second data pull
    mapper = pd.read_csv('/Users/ethanweiss/Desktop/Class documents/Spring 2020/Senior Design/IDmap.csv',header = None)
    header_dict = {}
    for (x,y) in zip(list(mapper.iloc[:,0]),list(mapper.iloc[:,1])):
        header_dict[x] = y
    data.rename(header_dict,axis = 1,inplace = True)
    # initialize dictionary for unreliable indecies
    unreliable_dict = {}
    # get indecies for all "eDNA Status as String" columns
    status_cols = [i for i in range(len(data.columns)) if i%3 == 2]
    # chiller_cols = [i for i in range(len(data.columns)) if i%3 == 0]
    # chiller_col_labels = [data.columns[x] for x in chiller_cols]

    # Loop through all status columns 
    for j in status_cols:
        unr_index = [] # reset unreliable index list to empty
        for i in range(len(data)): # loop through all rows of data
            if data.iloc[i,j] == 'UNRELIABLE': # check if unreliable
                unr_index.append(i) # if unreliable, add index to list
        unreliable_dict[j] = unr_index # assign list of unreliable indecies to column index key
        
    # DEBUG
    # Count number of UNRELIABLE points
    total = 0
    for x in unreliable_dict.keys():
        total += len(unreliable_dict[x])
    print('Total Unreliable = {0}.'.format(total))
        
    # initialize variables to protect against case of 1st point being unreliable
    before_bound = 0
    after_bound = 0
    before_index = 0
    # loop through all column indecies with unreliable data
    for x in unreliable_dict.keys():
        # loop through all unreliable values
        for i in unreliable_dict[x]:
            # loop through all indecies before unreliable point
            for j in range(1,i):
                if data.iloc[i-j,x] != 'UNRELIABLE': # check for unreliable point
                    # IF NOT UNRELIABLE: 
                    before_bound = data.iloc[i-j,x-1] # set before bound value
                    before_index = i-j # set before index value
                    break # break loop once before bound is found
            # loop through all indecies after unreliable point
            for j in range(1,len(data) - i): 
                if data.iloc[i+j,x] != 'UNRELIABLE': # check for unreliable point
                    # IF NOT UNRELIABLE:
                    after_bound = data.iloc[i+j,x-1] # set after bound value
                    after_index = i+j # set after index value
                    break # break loop once after bound is found

            # Linear Interpolation to substitute for UNRELIABLE values
            slope = (after_bound - before_bound)/(after_index - before_index)
            data.iloc[i,x-1] = before_bound + slope * (i - before_index)

    # Replace 'UNRELIABLE' tags with 'LI' tags
    for x in unreliable_dict.keys():
        for i in unreliable_dict[x]:
            data.iloc[i,x] = 'LI'
            
    # DEBUG
    # Count number of LI tags
    total2 = 0
    for j in status_cols:
        for i in range(len(data)):
            if data.iloc[i,j] == 'LI':
                total2 += 1
    # Check correct number of LI tags before saving to .csv
    try: 
        if total == total2:
            li_filepath = filepath.split('.csv')[0] + '_LItags.csv'
            data.to_csv(li_filepath,index = False)
            print('Data with LI tags saved to .csv.\n')
        else:
            raise ValueError('Total number of unreliable points does not equal total number of LI points.')
    except ValueError as e:
        print(e)
        return
    
    # Get chiller names
    # chiller_cols = [i for i in range(len(data.columns)) if i%3 == 0]
    # chiller_cols = [data.columns[x] for x in chiller_cols]

    # Get chiller names for second data pull
    chiller_cols = [i * 2+1 for i in range(int(len(data.columns)/2))]
    chiller_cols = [data.columns[x] for x in chiller_cols]

    # Get value columns to keep
    # value_cols = [i for i in range(len(data.columns)) if i%3 == 1]
    # value_cols = [data.columns[x] for x in value_cols]

    # Compile columns to keep in dataframe
    # data.rename({'AHSC_CHL01_ChlrPwr':'Date/Time'},axis = 1,inplace = True)
    # keep_cols = ['Date/Time'] + value_cols

    # Get keep cols for second data pull
    keep_cols = ['Date'] + chiller_cols

    # Get relevant subset of dataframe
    rel_data = data[keep_cols]
    # Create Dictionary for guide to rename columns with appropriate chiller names
    # rename_dict = {}
    # for i in range(len(keep_cols[1:])):
    #     rename_dict[keep_cols[i+1]] = chiller_cols[i]
    # # Assign new column names (appropriate chiller names)
    # rel_data.rename(rename_dict,axis = 1,inplace = True)
    
    
    # SAVE CSV
    try:
        rel_filepath = filepath.split('.csv')[0] + '_ChillerValuesOnly.csv'
        rel_data.to_csv(rel_filepath,index = False)
        print('Data of Chiller values only saved to .csv.')
    except Exception as e:
        print(e)

In [40]:
dataCleaning('/Users/ethanweiss/Desktop/Class documents/Spring 2020/Senior Design/Chiller Tons/TONS/CHRP201718.csv')

Total Unreliable = 53942.
Data with LI tags saved to .csv.

Data of Chiller values only saved to .csv.


## Notes on data from 2nd data pull:

Power data:

AHSC 2016 data has 183487 unreliable points. Probably not a usable data set. <br>

CHRP 2017-2018 has 203433 unreliable points. Probably not a usable data set. <br>
CHRP 2015-2016 has 292115 unreliable points. Probably not a usable data set. <br>

CRB 2016 has 521516 unreliable points. Probably not a usable data set. <br>


AHSC 2018 has 2951 unreliable points. CSV's successfully saved. <br>
AHSC 2017 has 318 unreliable points. CSV's successfully saved. <br>
AHSC 2015 has 5295 unreliable points. CSV's successfully saved. <br>

CRB 2018 has 12615 unreliable points. CSV's successfully saved. <br>
CRB 2017 has 17683 unreliable points. CSV's successfully saved. <br>
CRB 2015 has 9301 unreliable points. CSV's successfully saved. <br>

Tonnage data:

AHSC 2017-2018 has 104216 unreliable points. <br>

CHRP 2017-2018 has 53942 unreliable points. 